In [1]:
! pip install -q pandas matplotlib scikit-learn openai dotenv tqdm liqfit sentencepiece transformers

import sys
print("Current python version: ", sys.version)

import pandas as pd
import matplotlib.pyplot as plt
import re
import datasketch

Current python version:  3.11.11 (main, Dec 11 2024, 10:25:04) [Clang 14.0.6 ]


# Prepare supreme courts data for labeling 

In [91]:
# read parquet file
df = pd.read_csv('court_cases_labeled.csv')
df.head()

,address_count,address_occurrences,annotation_id,annotator,case,created_at,gender,id,information,information_count,information_occurrences,lead_time,number,number_count,number_occurrences,person_count,person_occurrences,sum_of_unique_entities,text,updated_at
0,1,"[{'start': 1844, 'end': 1852, 'text': 'АДРЕСА_...",51,1,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",2025-04-05T21:17:35.092054Z,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",121957732,"[{""end"":276,""text"":""ІНФОРМАЦІЯ_1"",""start"":264,...",6,"[{'start': 264, 'end': 276, 'text': 'ІНФОРМАЦІ...",180.598,"[{""end"":1824,""text"":""НОМЕР_1"",""start"":1817,""la...",1,"[{'start': 1817, 'end': 1824, 'text': 'НОМЕР_1'}]",1,"[{'start': 306, 'end': 313, 'text': 'ОСОБА_1'}]",4.0,УХВАЛА\n23 вересня 2024 року\nм. Київ\nсправа ...,2025-04-05T21:17:35.092073Z
1,2,"[{'start': 1334, 'end': 1342, 'text': 'АДРЕСА_...",58,1,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",2025-04-05T23:09:49.100238Z,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",124028233,"[{""end"":1970,""text"":""ІНФОРМАЦІЯ_1"",""start"":195...",1,"[{'start': 1958, 'end': 1970, 'text': 'ІНФОРМА...",168.785,"[{""end"":1359,""text"":""НОМЕР_1"",""start"":1352,""la...",2,"[{'start': 1352, 'end': 1359, 'text': 'НОМЕР_1...",8,"[{'start': 276, 'end': 283, 'text': 'ОСОБА_1'}...",4.0,УХВАЛА\n23 грудня 2024 року\nм. Київ\nсправа №...,2025-04-05T23:09:49.100258Z
2,2,"{'АДРЕСА_1': [(493, 501), (775, 783)]}",11,1,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",2025-03-27T18:44:06.616880Z,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",116850780,"[{""end"":473,""text"":""ІНФОРМАЦІЯ_1"",""start"":461,...",2,"{'ІНФОРМАЦІЯ_1': [(461, 473), (743, 755)]}",161.415,"[{""end"":518,""text"":""НОМЕР_1"",""start"":511,""labe...",4,"{'НОМЕР_2': [(677, 684), (915, 922)], 'НОМЕР_1...",3,"{'ОСОБА_1': [(277, 284), (451, 458), (733, 740)]}",NaN,Справа №487/573/24\nПровадження №2-н/487/512/2...,2025-03-28T10:33:53.781934Z
3,1,"[{'start': 2045, 'end': 2053, 'text': 'АДРЕСА_...",42,1,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",2025-04-05T20:35:46.845645Z,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",116574858,"[{""end"":891,""text"":""ІНФОРМАЦІЯ_1"",""start"":879,...",4,"[{'start': 2140, 'end': 2152, 'text': 'ІНФОРМА...",656.800,"[{""end"":2042,""text"":""НОМЕР_1"",""start"":2035,""la...",1,"[{'start': 2035, 'end': 2042, 'text': 'НОМЕР_1'}]",8,"[{'start': 869, 'end': 876, 'text': 'ОСОБА_3'}...",4.0,У х в а л а\n24 січня 2024 року\nм. Київ\nСпра...,2025-04-05T20:35:46.845657Z
4,2,"[{'start': 663, 'end': 671, 'text': 'АДРЕСА_1'...",61,1,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",2025-04-05T23:33:43.969199Z,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",117686034,"[{""end"":463,""text"":""ІНФОРМАЦІЯ_1"",""start"":451,...",6,"[{'start': 451, 'end': 463, 'text': 'ІНФОРМАЦІ...",409.074,"[{""end"":964,""text"":""НОМЕР_1"",""start"":957,""labe...",2,"[{'start': 620, 'end': 627, 'text': 'НОМЕР_1'}...",8,"[{'start': 269, 'end': 276, 'text': 'ОСОБА_1'}...",4.0,УХВАЛА\n14 березня 2024 року\nм. Київ\nсправа ...,2025-04-05T23:33:43.969220Z


In [92]:
# a function to migrate from {'НОМЕР_2': [(677, 684), (915, 922)], 'НОМЕР_1': [(511, 518), (793, 800)]} to [{'start': 1352, 'end': 1359, 'text': 'НОМЕР_1'}, {'start': 1980, 'end': 1987, 'text': 'НОМЕР_1'}]
def migrate_number_occurrences(occurrences):
    if isinstance(occurrences, str):
        occurrences = eval(occurrences)
    if isinstance(occurrences, dict):
        return [{'start': start, 'end': end, 'text': text} for text, positions in occurrences.items() for start, end in positions]
    return occurrences

df['number_occurrences'] = df['number_occurrences'].apply(migrate_number_occurrences)
df['person_occurrences'] = df['person_occurrences'].apply(migrate_number_occurrences)
df['address_occurrences'] = df['address_occurrences'].apply(migrate_number_occurrences)
df['information_occurrences'] = df['information_occurrences'].apply(migrate_number_occurrences)
df['sum_of_unique_entities'] = df.apply(lambda row: sum([row['number_count'] > 0,  row['information_count'] > 0, row['person_count'] > 0, row['address_count'] > 0]), axis=1)
df.head()

,address_count,address_occurrences,annotation_id,annotator,case,created_at,gender,id,information,information_count,information_occurrences,lead_time,number,number_count,number_occurrences,person_count,person_occurrences,sum_of_unique_entities,text,updated_at
0,1,"[{'start': 1844, 'end': 1852, 'text': 'АДРЕСА_...",51,1,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",2025-04-05T21:17:35.092054Z,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",121957732,"[{""end"":276,""text"":""ІНФОРМАЦІЯ_1"",""start"":264,...",6,"[{'start': 264, 'end': 276, 'text': 'ІНФОРМАЦІ...",180.598,"[{""end"":1824,""text"":""НОМЕР_1"",""start"":1817,""la...",1,"[{'start': 1817, 'end': 1824, 'text': 'НОМЕР_1'}]",1,"[{'start': 306, 'end': 313, 'text': 'ОСОБА_1'}]",4,УХВАЛА\n23 вересня 2024 року\nм. Київ\nсправа ...,2025-04-05T21:17:35.092073Z
1,2,"[{'start': 1334, 'end': 1342, 'text': 'АДРЕСА_...",58,1,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",2025-04-05T23:09:49.100238Z,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",124028233,"[{""end"":1970,""text"":""ІНФОРМАЦІЯ_1"",""start"":195...",1,"[{'start': 1958, 'end': 1970, 'text': 'ІНФОРМА...",168.785,"[{""end"":1359,""text"":""НОМЕР_1"",""start"":1352,""la...",2,"[{'start': 1352, 'end': 1359, 'text': 'НОМЕР_1...",8,"[{'start': 276, 'end': 283, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n23 грудня 2024 року\nм. Київ\nсправа №...,2025-04-05T23:09:49.100258Z
2,2,"[{'start': 493, 'end': 501, 'text': 'АДРЕСА_1'...",11,1,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",2025-03-27T18:44:06.616880Z,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",116850780,"[{""end"":473,""text"":""ІНФОРМАЦІЯ_1"",""start"":461,...",2,"[{'start': 461, 'end': 473, 'text': 'ІНФОРМАЦІ...",161.415,"[{""end"":518,""text"":""НОМЕР_1"",""start"":511,""labe...",4,"[{'start': 677, 'end': 684, 'text': 'НОМЕР_2'}...",3,"[{'start': 277, 'end': 284, 'text': 'ОСОБА_1'}...",4,Справа №487/573/24\nПровадження №2-н/487/512/2...,2025-03-28T10:33:53.781934Z
3,1,"[{'start': 2045, 'end': 2053, 'text': 'АДРЕСА_...",42,1,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",2025-04-05T20:35:46.845645Z,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",116574858,"[{""end"":891,""text"":""ІНФОРМАЦІЯ_1"",""start"":879,...",4,"[{'start': 2140, 'end': 2152, 'text': 'ІНФОРМА...",656.800,"[{""end"":2042,""text"":""НОМЕР_1"",""start"":2035,""la...",1,"[{'start': 2035, 'end': 2042, 'text': 'НОМЕР_1'}]",8,"[{'start': 869, 'end': 876, 'text': 'ОСОБА_3'}...",4,У х в а л а\n24 січня 2024 року\nм. Київ\nСпра...,2025-04-05T20:35:46.845657Z
4,2,"[{'start': 663, 'end': 671, 'text': 'АДРЕСА_1'...",61,1,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",2025-04-05T23:33:43.969199Z,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",117686034,"[{""end"":463,""text"":""ІНФОРМАЦІЯ_1"",""start"":451,...",6,"[{'start': 451, 'end': 463, 'text': 'ІНФОРМАЦІ...",409.074,"[{""end"":964,""text"":""НОМЕР_1"",""start"":957,""labe...",2,"[{'start': 620, 'end': 627, 'text': 'НОМЕР_1'}...",8,"[{'start': 269, 'end': 276, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n14 березня 2024 року\nм. Київ\nсправа ...,2025-04-05T23:33:43.969220Z


In [93]:
# validate no old duplicates by id
print("No duplicats by ID" if df['id'].duplicated().sum() == 0 else "Duplicates by ID")

No duplicats by ID


In [94]:
# Rename columns: case -> grammatical_case_labels, gender -> grammatical_gender_labels, number -> number_labels, information -> information_labels
df.rename(columns={'case': 'grammatical_case_labels', 'gender': 'grammatical_gender_labels', 'number': 'number_labels', 'information': 'information_labels'}, inplace=True)
df.head()


,address_count,address_occurrences,annotation_id,annotator,grammatical_case_labels,created_at,grammatical_gender_labels,id,information_labels,information_count,information_occurrences,lead_time,number_labels,number_count,number_occurrences,person_count,person_occurrences,sum_of_unique_entities,text,updated_at
0,1,"[{'start': 1844, 'end': 1852, 'text': 'АДРЕСА_...",51,1,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",2025-04-05T21:17:35.092054Z,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",121957732,"[{""end"":276,""text"":""ІНФОРМАЦІЯ_1"",""start"":264,...",6,"[{'start': 264, 'end': 276, 'text': 'ІНФОРМАЦІ...",180.598,"[{""end"":1824,""text"":""НОМЕР_1"",""start"":1817,""la...",1,"[{'start': 1817, 'end': 1824, 'text': 'НОМЕР_1'}]",1,"[{'start': 306, 'end': 313, 'text': 'ОСОБА_1'}]",4,УХВАЛА\n23 вересня 2024 року\nм. Київ\nсправа ...,2025-04-05T21:17:35.092073Z
1,2,"[{'start': 1334, 'end': 1342, 'text': 'АДРЕСА_...",58,1,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",2025-04-05T23:09:49.100238Z,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",124028233,"[{""end"":1970,""text"":""ІНФОРМАЦІЯ_1"",""start"":195...",1,"[{'start': 1958, 'end': 1970, 'text': 'ІНФОРМА...",168.785,"[{""end"":1359,""text"":""НОМЕР_1"",""start"":1352,""la...",2,"[{'start': 1352, 'end': 1359, 'text': 'НОМЕР_1...",8,"[{'start': 276, 'end': 283, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n23 грудня 2024 року\nм. Київ\nсправа №...,2025-04-05T23:09:49.100258Z
2,2,"[{'start': 493, 'end': 501, 'text': 'АДРЕСА_1'...",11,1,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",2025-03-27T18:44:06.616880Z,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",116850780,"[{""end"":473,""text"":""ІНФОРМАЦІЯ_1"",""start"":461,...",2,"[{'start': 461, 'end': 473, 'text': 'ІНФОРМАЦІ...",161.415,"[{""end"":518,""text"":""НОМЕР_1"",""start"":511,""labe...",4,"[{'start': 677, 'end': 684, 'text': 'НОМЕР_2'}...",3,"[{'start': 277, 'end': 284, 'text': 'ОСОБА_1'}...",4,Справа №487/573/24\nПровадження №2-н/487/512/2...,2025-03-28T10:33:53.781934Z
3,1,"[{'start': 2045, 'end': 2053, 'text': 'АДРЕСА_...",42,1,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",2025-04-05T20:35:46.845645Z,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",116574858,"[{""end"":891,""text"":""ІНФОРМАЦІЯ_1"",""start"":879,...",4,"[{'start': 2140, 'end': 2152, 'text': 'ІНФОРМА...",656.800,"[{""end"":2042,""text"":""НОМЕР_1"",""start"":2035,""la...",1,"[{'start': 2035, 'end': 2042, 'text': 'НОМЕР_1'}]",8,"[{'start': 869, 'end': 876, 'text': 'ОСОБА_3'}...",4,У х в а л а\n24 січня 2024 року\nм. Київ\nСпра...,2025-04-05T20:35:46.845657Z
4,2,"[{'start': 663, 'end': 671, 'text': 'АДРЕСА_1'...",61,1,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",2025-04-05T23:33:43.969199Z,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",117686034,"[{""end"":463,""text"":""ІНФОРМАЦІЯ_1"",""start"":451,...",6,"[{'start': 451, 'end': 463, 'text': 'ІНФОРМАЦІ...",409.074,"[{""end"":964,""text"":""НОМЕР_1"",""start"":957,""labe...",2,"[{'start': 620, 'end': 627, 'text': 'НОМЕР_1'}...",8,"[{'start': 269, 'end': 276, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n14 березня 2024 року\nм. Київ\nсправа ...,2025-04-05T23:33:43.969220Z


In [95]:
# print sum of nan in each column
print(df.isna().sum())
# replace nan with empty string
df = df.fillna('[]')


address_count                0
address_occurrences          0
annotation_id                0
annotator                    0
grammatical_case_labels      4
created_at                   0
grammatical_gender_labels    4
id                           0
information_labels           0
information_count            0
information_occurrences      0
lead_time                    0
number_labels                0
number_count                 0
number_occurrences           0
person_count                 0
person_occurrences           0
sum_of_unique_entities       0
text                         0
updated_at                   0
dtype: int64


In [96]:
df.head()

,address_count,address_occurrences,annotation_id,annotator,grammatical_case_labels,created_at,grammatical_gender_labels,id,information_labels,information_count,information_occurrences,lead_time,number_labels,number_count,number_occurrences,person_count,person_occurrences,sum_of_unique_entities,text,updated_at
0,1,"[{'start': 1844, 'end': 1852, 'text': 'АДРЕСА_...",51,1,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",2025-04-05T21:17:35.092054Z,"[{""end"":313,""text"":""ОСОБА_1"",""start"":306,""labe...",121957732,"[{""end"":276,""text"":""ІНФОРМАЦІЯ_1"",""start"":264,...",6,"[{'start': 264, 'end': 276, 'text': 'ІНФОРМАЦІ...",180.598,"[{""end"":1824,""text"":""НОМЕР_1"",""start"":1817,""la...",1,"[{'start': 1817, 'end': 1824, 'text': 'НОМЕР_1'}]",1,"[{'start': 306, 'end': 313, 'text': 'ОСОБА_1'}]",4,УХВАЛА\n23 вересня 2024 року\nм. Київ\nсправа ...,2025-04-05T21:17:35.092073Z
1,2,"[{'start': 1334, 'end': 1342, 'text': 'АДРЕСА_...",58,1,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",2025-04-05T23:09:49.100238Z,"[{""end"":626,""text"":""ОСОБА_1"",""start"":619,""labe...",124028233,"[{""end"":1970,""text"":""ІНФОРМАЦІЯ_1"",""start"":195...",1,"[{'start': 1958, 'end': 1970, 'text': 'ІНФОРМА...",168.785,"[{""end"":1359,""text"":""НОМЕР_1"",""start"":1352,""la...",2,"[{'start': 1352, 'end': 1359, 'text': 'НОМЕР_1...",8,"[{'start': 276, 'end': 283, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n23 грудня 2024 року\nм. Київ\nсправа №...,2025-04-05T23:09:49.100258Z
2,2,"[{'start': 493, 'end': 501, 'text': 'АДРЕСА_1'...",11,1,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",2025-03-27T18:44:06.616880Z,"[{""end"":284,""text"":""ОСОБА_1"",""start"":277,""labe...",116850780,"[{""end"":473,""text"":""ІНФОРМАЦІЯ_1"",""start"":461,...",2,"[{'start': 461, 'end': 473, 'text': 'ІНФОРМАЦІ...",161.415,"[{""end"":518,""text"":""НОМЕР_1"",""start"":511,""labe...",4,"[{'start': 677, 'end': 684, 'text': 'НОМЕР_2'}...",3,"[{'start': 277, 'end': 284, 'text': 'ОСОБА_1'}...",4,Справа №487/573/24\nПровадження №2-н/487/512/2...,2025-03-28T10:33:53.781934Z
3,1,"[{'start': 2045, 'end': 2053, 'text': 'АДРЕСА_...",42,1,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",2025-04-05T20:35:46.845645Z,"[{""end"":337,""text"":""ОСОБА_1"",""start"":330,""labe...",116574858,"[{""end"":891,""text"":""ІНФОРМАЦІЯ_1"",""start"":879,...",4,"[{'start': 2140, 'end': 2152, 'text': 'ІНФОРМА...",656.800,"[{""end"":2042,""text"":""НОМЕР_1"",""start"":2035,""la...",1,"[{'start': 2035, 'end': 2042, 'text': 'НОМЕР_1'}]",8,"[{'start': 869, 'end': 876, 'text': 'ОСОБА_3'}...",4,У х в а л а\n24 січня 2024 року\nм. Київ\nСпра...,2025-04-05T20:35:46.845657Z
4,2,"[{'start': 663, 'end': 671, 'text': 'АДРЕСА_1'...",61,1,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",2025-04-05T23:33:43.969199Z,"[{""end"":276,""text"":""ОСОБА_1"",""start"":269,""labe...",117686034,"[{""end"":463,""text"":""ІНФОРМАЦІЯ_1"",""start"":451,...",6,"[{'start': 451, 'end': 463, 'text': 'ІНФОРМАЦІ...",409.074,"[{""end"":964,""text"":""НОМЕР_1"",""start"":957,""labe...",2,"[{'start': 620, 'end': 627, 'text': 'НОМЕР_1'}...",8,"[{'start': 269, 'end': 276, 'text': 'ОСОБА_1'}...",4,УХВАЛА\n14 березня 2024 року\nм. Київ\nсправа ...,2025-04-05T23:33:43.969220Z


In [97]:
df[df['id'] == 116616462]

,address_count,address_occurrences,annotation_id,annotator,grammatical_case_labels,created_at,grammatical_gender_labels,id,information_labels,information_count,information_occurrences,lead_time,number_labels,number_count,number_occurrences,person_count,person_occurrences,sum_of_unique_entities,text,updated_at
12,1,"[{'start': 2337, 'end': 2345, 'text': 'АДРЕСА_...",57,1,"[{""end"":572,""text"":""ОСОБА_1"",""start"":565,""labe...",2025-04-05T22:10:17.990887Z,"[{""end"":572,""text"":""ОСОБА_1"",""start"":565,""labe...",116616462,"[{""end"":617,""text"":""ІНФОРМАЦІЯ_2"",""start"":605,...",1,"[{'start': 605, 'end': 617, 'text': 'ІНФОРМАЦІ...",224.595,"[{""end"":295,""text"":""НОМЕР_1"",""start"":288,""labe...",8,"[{'start': 288, 'end': 295, 'text': 'НОМЕР_1'}...",1,"[{'start': 565, 'end': 572, 'text': 'ОСОБА_1'}]",4,УХВАЛА\n29 січня 2024 року\nм. Київ\nсправа №1...,2025-04-06T10:48:37.248647Z


In [99]:
def print_unmatching_list_sizes(row, label_column, occurrences_column):
    len_of_entities = len(row[occurrences_column])
    len_of_labels = len(eval(row[label_column]))

    if len_of_entities != len_of_labels:
        print(f'{row["id"]:<20}: {len_of_entities:<20} != {len_of_labels}')
        # sort by start position
        print(len(row[occurrences_column]))
        # sort by start position
        print(len(eval(row[label_column])))
        print(row[label_column])
        print(row[occurrences_column])

print('Grammatical case labels and occurrences:')
for index, row in df.iterrows():
    print_unmatching_list_sizes(row, "grammatical_case_labels", "person_occurrences")

print('--------------------------------')
print('--------------------------------')


print('Grammatical gender labels and occurrences:')
for index, row in df.iterrows():
    print_unmatching_list_sizes(row, "grammatical_gender_labels", "person_occurrences")
    
print('--------------------------------')
print('--------------------------------')

print('Number labels and occurrences:')
for index, row in df.iterrows():
    print_unmatching_list_sizes(row, "number_labels", "number_occurrences")

print('--------------------------------')
print('--------------------------------')

print('Information labels and occurrences:')
for index, row in df.iterrows():
    print_unmatching_list_sizes(row, "information_labels", "information_occurrences")


Grammatical case labels and occurrences:
--------------------------------
--------------------------------
Grammatical gender labels and occurrences:
--------------------------------
--------------------------------
Number labels and occurrences:
--------------------------------
--------------------------------
Information labels and occurrences:


In [100]:
# save df to parquet
df.to_parquet('court_cases_labeled.parquet')